Test de estres. El objetivo es evaluar el modelo bajo tres escenarios: Sin oclusion, oclusion moderada (parches del 25%) y oclusion severa (parches 50%)

In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os
from google.colab import drive

In [2]:
#Cargar el modelo entrenado
#1. Configuracion
drive.mount('/content/drive')

DATA_DIR = '/content/drive/MyDrive/Master Thesis/Data/Split_Dataset'
Save_Path = '/content/drive/MyDrive/Master Thesis/Models/vit_archivtecture_model.pth'

#Crear carpeta de modelos si no existe
os.makedirs(os.path.dirname(Save_Path), exist_ok=True)

#Configuracion de dispositivos
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Ejectuando en: {device}")

Mounted at /content/drive
Ejectuando en: cuda


In [3]:
#Definir transformaciones y dataloader
val_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_dataset = datasets.ImageFolder(os.path.join(DATA_DIR, 'val'), val_transforms)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
class_names = val_dataset.classes
num_classes = len(class_names)

print(f"Dataset Cargado. Evaluando {len(val_dataset)} imagenes de {num_classes} clases.")

Dataset Cargado. Evaluando 1352 imagenes de 14 clases.


In [4]:
#Reconstruir y cargar el modelo de 327MB
model = models.vit_b_16()
model.heads.head = nn.Linear(model.heads.head.in_features, num_classes)

if os.path.exists(Save_Path):
    model.load_state_dict(torch.load(Save_Path))
    model.to(device)
    model.eval()
    print("Modelo cargado existosamente desde Drive")
else:
    print("No se encontró el archivo de modelo en Drive")


Modelo cargado existosamente desde Drive


In [5]:
# Funcion de oclusion
def aplicar_oclusion_batch(batch_tensors, ratio=0.25):
  ocluded_batch = batch_tensors.clone()
  n, c, h, w = ocluded_batch.shape
  patch_size = int(h * ratio)

  for i in range(n):
    y = torch.randint(0, h - patch_size, (1,))
    x = torch.randint(0, w - patch_size, (1,))
    ocluded_batch[i, :, y:y+patch_size, x:x+patch_size] = 0
  return ocluded_batch

In [8]:
#Test de estres masivo
niveles = [0.0, 0.25, 0.50]
resultados = {}

for nivel in niveles:
  correctos = 0
  total = 0
  print(f"Testing oclusion al {nivel*100:.0f}%...")

  with torch.no_grad():
    for inputs, labels in val_loader:
      inputs, labels = inputs.to(device), labels.to(device)
      if nivel > 0:
        inputs = aplicar_oclusion_batch(inputs, nivel)

      outputs = model(inputs)
      _, preds = torch.max(outputs, 1)
      correctos += torch.sum(preds == labels.data)
      total += labels.size(0)

  accuracy = correctos.double() / total
  resultados[nivel] = accuracy.item()

Testing oclusion al 0%...
Testing oclusion al 25%...
Testing oclusion al 50%...


In [9]:
#Resultados finales para tu tests
print("\n" + "="*30)
print("RESULTADOS DE VISIBILIDAD PARCIAL")
print("="*30)
for nivel, acc in resultados.items():
  print(f"Oclusion {nivel*100:>3.0f}% | Accuracy: {acc:.4f}")


RESULTADOS DE VISIBILIDAD PARCIAL
Oclusion   0% | Accuracy: 0.9283
Oclusion  25% | Accuracy: 0.9075
Oclusion  50% | Accuracy: 0.8550
